In [ ]:
import os
import random
import numpy as np
from skimage import color
from tqdm.notebook import tqdm
from PIL import Image, ImageEnhance

from lib.lime import LIME
from lib.fog_gen import generate_fog

In [ ]:
class FogEffectGenerator:
    def __init__(self):
        self._lime = LIME(iterations=25, alpha=1.0)
        self._illumination2darkness = {0: 1, 1: 0.75, 2: 0.65, 3:0.5}
        self._weather2visibility = {'fog': (100,250), 'rain': (1000,2000), 'snow': (500, 1000)}
        self._illumination2fogcolor = {0: (80, 120), 1: (120, 160), 2: (160, 200), 3: (200, 240)}
    
    def getIlluminationMap(self, img: np.ndarray) -> np.ndarray: 
        self._lime.load(img)
        T = self._lime.illumMap()
        return T
    
    @staticmethod
    def reduce_lightHSV(rgb, sat_red=0.5, val_red=0.5):
        hsv = color.rgb2hsv(rgb/255)
        hsv[...,1] *= sat_red
        hsv[...,2] *= val_red
        return (color.hsv2rgb(hsv)*255).astype(np.uint8)
    
    @staticmethod
    def fogAttenuation(img: np.ndarray, depth:np.ndarray, visibility=1000, fog_color=200):
        img_fog = generate_fog(img.copy(), depth.copy(), visibility=visibility, fog_color=fog_color)
        return img_fog
    
    def genEffect(self, img_path: str, depth_path: str, weather_type='fog'):
        I = np.array(Image.open(img_path))
        D = np.load(depth_path)
        
        T = self.getIlluminationMap(I)
        illumination_array = np.histogram(T, bins=4, range=(0,1))[0]/(T.size)
        illumination = illumination_array.argmax()
        
        if illumination>0:
            vmax = self._weather2visibility[weather_type][1] if self._weather2visibility[weather_type][1]<=D.max() else D.max()
            visibility = random.randint(self._weather2visibility[weather_type][0], int(vmax))
            fog_color = random.randint(self._illumination2fogcolor[illumination][0], self._illumination2fogcolor[illumination][1])
            I_dark = self.reduce_lightHSV(I, sat_red=self._illumination2darkness[illumination], val_red=self._illumination2darkness[illumination])
            I_fog = self.fogAttenuation(I_dark, D, visibility=visibility, fog_color=fog_color)
        else:
            fog_color = 75
            visibility = 150 #D.max()*0.75
            I_fog = self.fogAttenuation(I, D, visibility=visibility, fog_color=fog_color)
            
        return I_fog

In [ ]:
foggen = FogEffectGenerator()

In [ ]:
save_folder = "../../dataset/train/fog_synthetic/images"
depth_folder = "fog_depth"

with open("fog_list.txt", "r") as f:
    content_files = f.read().splitlines() 
content_files = [Path(p) for p in content_files]

for path in tqdm(content_files):
    foggy = foggen.genEffect(path, os.path.join(depth_folder, path.stem+".npy"))
    Image.fromarray(foggy.astype(np.uint8)).save(os.path.join(save_folder, path.stem+"_fsyn.jpg"))

In [ ]:
img_name = "b1d3907b-2278601b"
image_path = f"data/rgb/{img_name}.jpg"
depth_path = f"data/depth/{img_name}.npy"

In [ ]:
I = we.genEffect(image_path, depth_path, "fog")

In [ ]:
Image.fromarray(I[0])

In [ ]:
I[1:]

In [ ]:
# self._illumination_type = {0: "dark", 1: "slightly dark", 2: "slightly bright", 3: "bright"}
# hist, bin_edges = np.histogram(T, bins=4, range=(0,1))
# illumination = self._illumination_type[hist.argmax()]

In [ ]:
weather2visibility = {'fog': (100,750), 
                      'rain': (1000,2000), 
                      'snow': (500, 1000)}

In [ ]:
random.randint(weather2visibility['fog'][0], weather2visibility['fog'][1])

In [ ]:
weather2visibility = {'fog': (100,250), 'rain': (1000,2000), 'snow': (500, 1000)}